In [20]:
import gym
import random
env = gym.make('CartPole-v1')
states=env.observation_space.shape[0]
actions=env.action_space.n

In [21]:



episodes=10
for episode in range(1,episodes+1):
   state=env.reset()
   score=0
   done=False
   
   while not done:
      env.render()
      action=random.choice([0,1])
      
      n_state,reward,done,info=env.step(action)
      score+=reward
   print(f'Episode :{episode}  score:{score}')
state=env.close()
   



Episode :1  score:11.0
Episode :2  score:27.0
Episode :3  score:18.0
Episode :4  score:10.0
Episode :5  score:20.0
Episode :6  score:30.0
Episode :7  score:17.0
Episode :8  score:20.0
Episode :9  score:17.0
Episode :10  score:9.0


In [25]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow

In [26]:
def buil_model(states,actions):
    model=tensorflow.keras.Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [27]:
model=buil_model(states,actions)

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [29]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [30]:
def build_agent(model,actions):
    policy=BoltzmannQPolicy()
    memory=SequentialMemory(limit=50000,window_length=1)
    dqn=DQNAgent(model=model,memory=memory,policy=policy,
                 nb_actions=actions,nb_steps_warmup=10,target_model_update=1e-2)
    return dqn

In [31]:
dqn=build_agent(model,actions)
dqn.compile(Adam(lr=1e-3),metrics=['mae'])
dqn.fit(env,nb_steps=50000,visualize=False,verbose=1)

C:\Users\abder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 20:56 - reward: 1.0000

C:\Users\abder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\abder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 102s 10ms/step - reward: 1.0000
104 episodes - episode_reward: 95.596 [9.000, 312.000] - loss: 2.835 - mae: 20.592 - mean_q: 41.728

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 91s 9ms/step - reward: 1.0000
41 episodes - episode_reward: 241.220 [180.000, 500.000] - loss: 5.687 - mae: 43.355 - mean_q: 87.708

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 98s 10ms/step - reward: 1.0000
34 episodes - episode_reward: 285.618 [214.000, 473.000] - loss: 5.563 - mae: 51.314 - mean_q: 103.698

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 111s 11ms/step - reward: 1.0000
40 episodes - episode_reward: 257.125 [190.000, 489.000] - loss: 3.986 - mae: 51.654 - mean_q: 104.092

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 111s 11ms/step - reward: 1.0000
done, took 512.239 seconds


In [32]:
scores=dqn.test(env,nb_episodes=100,visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 214.000, steps: 214
Episode 2: reward: 339.000, steps: 339
Episode 3: reward: 211.000, steps: 211
Episode 4: reward: 290.000, steps: 290
Episode 5: reward: 235.000, steps: 235
Episode 6: reward: 244.000, steps: 244
Episode 7: reward: 198.000, steps: 198
Episode 8: reward: 305.000, steps: 305
Episode 9: reward: 241.000, steps: 241
Episode 10: reward: 249.000, steps: 249
Episode 11: reward: 293.000, steps: 293
Episode 12: reward: 239.000, steps: 239
Episode 13: reward: 265.000, steps: 265
Episode 14: reward: 215.000, steps: 215
Episode 15: reward: 297.000, steps: 297
Episode 16: reward: 239.000, steps: 239
Episode 17: reward: 265.000, steps: 265
Episode 18: reward: 231.000, steps: 231
Episode 19: reward: 242.000, steps: 242
Episode 20: reward: 280.000, steps: 280
Episode 21: reward: 244.000, steps: 244
Episode 22: reward: 219.000, steps: 219
Episode 23: reward: 240.000, steps: 240
Episode 24: reward: 289.000, steps: 289
Episode 25: reward: 

In [33]:
scores=dqn.test(env,nb_episodes=5,visualize=True)
state=env.close()


Testing for 5 episodes ...
Episode 1: reward: 284.000, steps: 284
Episode 2: reward: 358.000, steps: 358
Episode 3: reward: 224.000, steps: 224
Episode 4: reward: 232.000, steps: 232
Episode 5: reward: 262.000, steps: 262


In [38]:
dqn.save_weights('./Training/Saved Models/dqn_weights.h5f',overwrite=True)

NameError: name 'dqn' is not defined

In [35]:
del model
del dqn
del env